# **Variant Filtering tutorial**

In this Jupyter tutorial you can learn how to query variants using PyOskar, which is one of the most typical use cases. As you will see PyOskar implements a rich and comprenhesive API to help you to query variants using the variant annotation, genotypes, file metrics, ...

In order to optimise the performance and be compatible with PySpark we decided to extend the standard PySpark API. We made this by using Spark UDF (User Defined Functions) implemented in Java. For instance, after creating a DataFrame using PyOskar you can use PyOskar filters such as _genes_ or _ct_ in PySpark, which will return one or more particular fields from that DataFrame.

First, we need to import the PyOskar and PySpark modules. Second, we need to create an instance of the _Oskar_ object, from which depends a big part of the functionality. Finally, we can load our data in a DataFrame _df_ and we are ready to start playing.

In [23]:
from pyoskar.core import Oskar
from pyoskar.sql import *
from pyoskar.analysis import *
from pyspark.sql.functions import col, udf, count, explode, concat, when, expr
from pyspark.sql.functions import *

oskar = Oskar(spark)
df = oskar.load("./data/platinum_chr22.small.parquet")

## Next instruction is needed for querying using Spark SQL language.
df.createOrReplaceTempView("platinum")

You can use PySpark to print the data from _df_. This is how our testing dataframe looks like, you can see that for this tutorial we are using a small dataset from Illumina Platinum Genomes with 1,000 random variants from chromosome 22:

In [3]:
print("Print first 20 variants:")
df.show()

Print first 20 variants:
+---------------+-----+----------+--------+--------+---------+---------+------+----+------+-----+----+--------------------+--------------------+
|             id|names|chromosome|   start|     end|reference|alternate|strand|  sv|length| type|hgvs|             studies|          annotation|
+---------------+-----+----------+--------+--------+---------+---------+------+----+------+-----+----+--------------------+--------------------+
|22:16054454:C:T|   []|        22|16054454|16054454|        C|        T|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 16054454, 16...|
|22:16065809:T:C|   []|        22|16065809|16065809|        T|        C|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 16065809, 16...|
|22:16077310:T:A|   []|        22|16077310|16077310|        T|        A|     +|null|     1|  SNV|  []|[[hgvauser@platin...|[22, 16077310, 16...|
|22:16080499:A:G|   []|        22|16080499|16080499|        A|        G|     +|null|     1|  SNV|  []|[[h

In [4]:
print("Total number of variants:")
df.count()

Total number of variants:


1000

## Region
In the first place of all we will execute a simple filtering based on a restricted zone for those people who are not familiarizes with spark syntax. In this example we chose the 22th chromosome and the nucleotides between 17.000.000 and 17.500.000 position. There is two main ways of querying our filter:

 - PySpark API

In [11]:
# PySpark - df
df.select("id", "chromosome", "start", "end").filter(df.chromosome == 22).filter(df.start > 17000000).filter(df.end < 17500000).show(10)

+---------------+----------+--------+--------+
|             id|chromosome|   start|     end|
+---------------+----------+--------+--------+
|22:17001352:C:G|        22|17001352|17001352|
|22:17002352:C:A|        22|17002352|17002352|
|22:17004097:G:A|        22|17004097|17004097|
|22:17011943:G:C|        22|17011943|17011943|
|22:17012760:G:A|        22|17012760|17012760|
|22:17013084:C:T|        22|17013084|17013084|
|22:17013900:A:G|        22|17013900|17013900|
|22:17019574:C:A|        22|17019574|17019574|
|22:17030949:T:C|        22|17030949|17030949|
|22:17034810:T:C|        22|17034810|17034810|
+---------------+----------+--------+--------+
only showing top 10 rows



 - Spark SQL

In [6]:
# Spark SQL - platinum
spark.sql("SELECT id, chromosome, start, end FROM platinum WHERE chromosome =='22' AND start > 17000000 AND end < 17500000").show(10)

+---------------+----------+--------+--------+
|             id|chromosome|   start|     end|
+---------------+----------+--------+--------+
|22:17001352:C:G|        22|17001352|17001352|
|22:17002352:C:A|        22|17002352|17002352|
|22:17004097:G:A|        22|17004097|17004097|
|22:17011943:G:C|        22|17011943|17011943|
|22:17012760:G:A|        22|17012760|17012760|
|22:17013084:C:T|        22|17013084|17013084|
|22:17013900:A:G|        22|17013900|17013900|
|22:17019574:C:A|        22|17019574|17019574|
|22:17030949:T:C|        22|17030949|17030949|
|22:17034810:T:C|        22|17034810|17034810|
+---------------+----------+--------+--------+
only showing top 10 rows



## Gene
We may want to execute a filtering which ties up the variants attached to a particular gene. NBEAP3 was chosen as the target. Here we start preciating the functionality of PyOskar API: [<span style="color:#FF1493"> **genes** </span>] will automatically locate and acces the field with that same name. We just need to specify the DataFrame column where we store the gene info: _annotation_. Now it's only left to define the filter.
<br>
Usage:
```python
genes(annotation[str])
```

In [10]:
# PySpark - df
df.select("id", genes("annotation").alias("genes")).filter(array_contains("genes", "NBEAP3")).show(truncate = False)

+---------------+----------------------+
|id             |genes                 |
+---------------+----------------------+
|22:16096040:G:A|[NBEAP3]              |
|22:16099957:C:T|[NBEAP3]              |
|22:16100462:A:G|[NBEAP3]              |
|22:16105660:G:A|[NBEAP3]              |
|22:16112391:G:A|[NBEAP3]              |
|22:16114913:A:T|[NBEAP3]              |
|22:16127471:A:-|[LA16c-60H5.7, NBEAP3]|
+---------------+----------------------+



Here is how we should use our genes function with pure Spark SQL:

In [7]:
# Spark SQL - platinum
spark.sql("SELECT id,genes(annotation) AS genes FROM platinum WHERE array_contains(genes(annotation), 'NBEAP3')").show(truncate = False)

+---------------+----------------------+
|id             |genes                 |
+---------------+----------------------+
|22:16096040:G:A|[NBEAP3]              |
|22:16099957:C:T|[NBEAP3]              |
|22:16100462:A:G|[NBEAP3]              |
|22:16105660:G:A|[NBEAP3]              |
|22:16112391:G:A|[NBEAP3]              |
|22:16114913:A:T|[NBEAP3]              |
|22:16127471:A:-|[LA16c-60H5.7, NBEAP3]|
+---------------+----------------------+



And ere we will introduce you a third form of building our query, which comes to be half PySpark - half Spark SQL:

In [8]:
# Spark SQL / PySpark
df.selectExpr("id", "genes(annotation) AS genes").filter("array_contains(genes, 'NBEAP3')").show(truncate = False)

+---------------+----------------------+
|id             |genes                 |
+---------------+----------------------+
|22:16096040:G:A|[NBEAP3]              |
|22:16099957:C:T|[NBEAP3]              |
|22:16100462:A:G|[NBEAP3]              |
|22:16105660:G:A|[NBEAP3]              |
|22:16112391:G:A|[NBEAP3]              |
|22:16114913:A:T|[NBEAP3]              |
|22:16127471:A:-|[LA16c-60H5.7, NBEAP3]|
+---------------+----------------------+



NOTE: Later in our tutorials we will only show the purest Pyspark syntax, but there would be no problem at all if we prefered to use Pyspark functionality with SQL enouncements.

## Genotype
In case we want to get the genotypes corresponding to any concrete sample we could use the [<span style="color:#FF1493"> **genotype** </span>] function. We will need to specify the field where that info is contained (_studies_), as well as the sample.
<br>
Usage:
```python
genotype(studies[str], sample[str])
```

In [12]:
df.select("id", genotype("studies", "NA12877").alias("NA12877")).show(10)

+---------------+-------+
|             id|NA12877|
+---------------+-------+
|22:16054454:C:T|    ./.|
|22:16065809:T:C|    0/1|
|22:16077310:T:A|    ./.|
|22:16080499:A:G|    0/1|
|22:16084621:T:C|    ./.|
|22:16091610:G:T|    ./.|
|22:16096040:G:A|    ./.|
|22:16099957:C:T|    0/1|
|22:16100462:A:G|    0/1|
|22:16105660:G:A|    0/1|
+---------------+-------+
only showing top 10 rows



We could also use a more generic function like [<span style="color:#FF1493"> **samples_data_field** </span>]. This method allow us to get any format field we desire.
<br>
Usage:
```python
samples_data(studies[str], annotation[str])
```

In [13]:
df.select(df.id, sample_data_field("studies", "NA12878", "GT").alias("NA12878")).show(10)

+---------------+-------+
|             id|NA12878|
+---------------+-------+
|22:16054454:C:T|    ./.|
|22:16065809:T:C|    ./.|
|22:16077310:T:A|    0/1|
|22:16080499:A:G|    ./.|
|22:16084621:T:C|    0/1|
|22:16091610:G:T|    ./.|
|22:16096040:G:A|    ./.|
|22:16099957:C:T|    0/1|
|22:16100462:A:G|    0/1|
|22:16105660:G:A|    ./.|
+---------------+-------+
only showing top 10 rows



And also we dispose of [<span style="color:#FF1493"> **samples_data** </span>], which will return all the format fields all in once. Unluckily this testing dataframe only stores genotype data!
<br>
Usage:
```python
samples_data_field(studies[str], annotation[str], formatField[str])
```

In [9]:
df.select(df.id, sample_data("studies", "NA12879").alias("NA12879")).show(10)

+---------------+-------+
|             id|NA12879|
+---------------+-------+
|22:16054454:C:T|  [./.]|
|22:16065809:T:C|  [./.]|
|22:16077310:T:A|  [0/1]|
|22:16080499:A:G|  [0/1]|
|22:16084621:T:C|  [./.]|
|22:16091610:G:T|  [./.]|
|22:16096040:G:A|  [./.]|
|22:16099957:C:T|  [0/1]|
|22:16100462:A:G|  [0/1]|
|22:16105660:G:A|  [0/1]|
+---------------+-------+
only showing top 10 rows



## Population Frequency
In order to get the population frequency data we may want to use the [<span style="color:#FF1493"> **population_frequency** </span>] UDF. Apart from specifying the field were we contain that information (_annotation_), we will need to specify the ID of the particular study we would like to check, the target population and a specific cohort.
<br>
Usage:
```python
population_frequency(annotation[str], study[str], population[str])
```

In [14]:
df.select("id", population_frequency("annotation", "GNOMAD_GENOMES", "ALL").alias("GNOMAD_GENOMES:ALL"))\
    .filter(population_frequency("annotation", "GNOMAD_GENOMES", "ALL") != 0).show(10, truncate = False)

+---------------+---------------------+
|id             |GNOMAD_GENOMES:ALL   |
+---------------+---------------------+
|22:16054454:C:T|0.07566695660352707  |
|22:16065809:T:C|0.14594951272010803  |
|22:16077310:T:A|0.2338419109582901   |
|22:16091610:G:T|0.0031298904214054346|
|22:16099957:C:T|0.6782668232917786   |
|22:16105660:G:A|0.12387744337320328  |
|22:16114913:A:T|0.11458797007799149  |
|22:16127471:A:-|0.004762233234941959 |
|22:16134019:G:T|0.03758351877331734  |
|22:16138943:C:G|0.2861359417438507   |
+---------------+---------------------+
only showing top 10 rows



Otherwise, if we are interested in getting all the Population Frequencies available independently of the cohort, we can use this other method named [<span style="color:#FF1493"> **population_frequency_as_map** </span>] which will return the whole set of PF as a dictionary format. Then we could apply "explode" and convert that dictionary into a new dataframe. In this example we have also filtered by variant to get a better looking output:
<br>
Usage:
```python
population_frequency_as_map(annotation[str])
```

In [7]:
PF = df.select(df.id, population_frequency_as_map("annotation").alias("population_frequencies")).filter(df.id == "22:16099957:C:T")
PF.select("id", explode(PF.population_frequencies).alias("study", "population_frequencies_df")).show(truncate = False)

+---------------+---------------------+-------------------------+
|id             |study                |population_frequencies_df|
+---------------+---------------------+-------------------------+
|22:16099957:C:T|GNOMAD_GENOMES:OTH   |0.6896551847457886       |
|22:16099957:C:T|GNOMAD_GENOMES:ALL   |0.6782668232917786       |
|22:16099957:C:T|GNOMAD_GENOMES:AFR   |0.6747621297836304       |
|22:16099957:C:T|GNOMAD_GENOMES:NFE   |0.6699579954147339       |
|22:16099957:C:T|GNOMAD_GENOMES:MALE  |0.682662844657898        |
|22:16099957:C:T|GNOMAD_GENOMES:FIN   |0.6726332306861877       |
|22:16099957:C:T|GNOMAD_GENOMES:EAS   |0.9523809552192688       |
|22:16099957:C:T|GNOMAD_GENOMES:ASJ   |0.5721649527549744       |
|22:16099957:C:T|GNOMAD_GENOMES:FEMALE|0.6727412939071655       |
|22:16099957:C:T|GNOMAD_GENOMES:AMR   |0.6950549483299255       |
+---------------+---------------------+-------------------------+



## Biotype
We also dispose of [<span style="color:#FF1493"> **biotypes** </span>] access, which looks like this:
<br>
Usage:
```python
biotypes(annotation[str])
```

In [19]:
df.select(df.id, biotypes("annotation").alias("biotypes")).filter(array_contains("biotypes", "protein_coding")).show(10, truncate = False)

+---------------+-------------------------------------------------------------------------------------+
|id             |biotypes                                                                             |
+---------------+-------------------------------------------------------------------------------------+
|22:16252532:A:G|[processed_pseudogene, protein_coding, nonsense_mediated_decay]                      |
|22:16289841:A:-|[protein_coding, nonsense_mediated_decay]                                            |
|22:17069538:C:A|[protein_coding]                                                                     |
|22:17270265:T:C|[protein_coding]                                                                     |
|22:17271137:T:-|[protein_coding]                                                                     |
|22:17282666:G:A|[protein_coding]                                                                     |
|22:17285049:G:C|[protein_coding]                               

## Consequence type
There are two ways we could acces _consequence type_ field. The first one is the [<span style="color:#FF1493"> **consequence_types** </span>] UDF:
<br>
Usage:
```python
consequence_type(annotation[str])
```

In [14]:
df.select(df.id, genes("annotation").alias("genes"), consequence_types("annotation").alias("consequence_types")).show(20, truncate = False)

+---------------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|id             |genes                                  |consequence_types                                                                                                      |
+---------------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|22:16054454:C:T|[]                                     |[intergenic_variant]                                                                                                   |
|22:16065809:T:C|[LA16c-4G1.3]                          |[regulatory_region_variant, downstream_gene_variant]                                                                   |
|22:16077310:T:A|[LA16c-4G1.4]                          |[2KB_upstream_variant, regulatory_region_variant]    

The [<span style="color:#FF1493"> **consequence_types_by_gene** </span>] method, unlike the one above, will return the field that contains the consequence types that **directly match the specified gene!!!**
<br>
Usage:
```python
consequence_type_as_map(annotation[str], gene[str])
```

In [22]:
df.select(df.id, genes("annotation").alias("genes"), consequence_types_by_gene("annotation", "NBEAP3").alias("consequence_types")).show(20, truncate = False)

+---------------+---------------------------------------+-----------------------------------------------+
|id             |genes                                  |consequence_types                              |
+---------------+---------------------------------------+-----------------------------------------------+
|22:16054454:C:T|[]                                     |[]                                             |
|22:16065809:T:C|[LA16c-4G1.3]                          |[]                                             |
|22:16077310:T:A|[LA16c-4G1.4]                          |[]                                             |
|22:16080499:A:G|[LA16c-4G1.4, LA16c-4G1.5]             |[]                                             |
|22:16084621:T:C|[LA16c-4G1.5]                          |[]                                             |
|22:16091610:G:T|[]                                     |[]                                             |
|22:16096040:G:A|[NBEAP3]                     

We may want to combine that output with a filter like this:

In [11]:
df.select(df.id, consequence_types_by_gene("annotation", "NBEAP3").alias("consequence_types"))\
    .filter(array_contains("consequence_types", "intron_variant")).show(10, truncate = False)

+---------------+-----------------------------------------------+
|id             |consequence_types                              |
+---------------+-----------------------------------------------+
|22:16105660:G:A|[non_coding_transcript_variant, intron_variant]|
|22:16112391:G:A|[non_coding_transcript_variant, intron_variant]|
|22:16114913:A:T|[non_coding_transcript_variant, intron_variant]|
+---------------+-----------------------------------------------+



## Protein substitution score
Finally, this is how we could filter by _conservation_, _functional_ or _protein substitution_ scores:

 - Conservation scores [<span style="color:#FF1493"> **conservation** </span>]: _gerp_, _phylop_ and _phastCons_

Usage:
```python
consequence_type_as_map(annotation[str], source[str])
```

In [20]:
df.select("id", conservation("annotation", "gerp").alias("gerp"), conservation("annotation", "phastCons").alias("phastCons")).filter("gerp > 0.5")\
    .filter("phastCons > 0.2").show(10, truncate = False)

+---------------+------------------+-------------------+
|id             |gerp              |phastCons          |
+---------------+------------------+-------------------+
|22:16127471:A:-|1.3799999952316284|0.2709999978542328 |
|22:18406473:C:T|1.7799999713897705|0.5329999923706055 |
|22:18824362:C:T|1.850000023841858 |0.4050000011920929 |
|22:18910451:A:C|0.9670000076293945|0.8029999732971191 |
|22:19018034:G:A|0.7429999709129333|0.3630000054836273 |
|22:21600633:C:G|1.0099999904632568|0.6600000262260437 |
|22:21606270:G:A|1.5800000429153442|0.2750000059604645 |
|22:21842655:C:T|1.3700000047683716|0.6650000214576721 |
|22:21906285:C:T|0.7689999938011169|0.37299999594688416|
|22:22172155:G:A|0.675000011920929 |0.30799999833106995|
+---------------+------------------+-------------------+
only showing top 10 rows



 - Functional scores [<span style="color:#FF1493"> **functional** </span>]: *cadd_scaled* and *cadd_raw*

Usage:
```python
functional(annotation[str], source[str])
```

In [21]:
df.select("id", functional("annotation", "cadd_scaled").alias("cadd_scaled"), functional("annotation", "cadd_raw").alias("cadd_raw")).filter("cadd_raw < 0").show(10)

+---------------+-------------------+--------------------+
|             id|        cadd_scaled|            cadd_raw|
+---------------+-------------------+--------------------+
|22:16054454:C:T| 1.5700000524520874|-0.13000011444091797|
|22:16065809:T:C| 1.4299999475479126|-0.14999961853027344|
|22:16080499:A:G| 0.7900000214576721| -0.2700004577636719|
|22:16084621:T:C|0.44999998807907104|-0.36999988555908203|
|22:16096040:G:A|  0.949999988079071|-0.22999954223632812|
|22:16112391:G:A|0.03999999910593033| -0.7799997329711914|
|22:16114913:A:T| 0.8700000047683716|               -0.25|
|22:16134019:G:T|0.49000000953674316| -0.3599996566772461|
|22:16144239:T:C|               1.25|-0.18000030517578125|
|22:16229483:C:T|0.03999999910593033| -0.8100004196166992|
+---------------+-------------------+--------------------+
only showing top 10 rows



 - Protein substitution scores [<span style="color:#FF1493"> **protein_substitution** </span>]: _polyphen_ and _sift_

Usage:
```python
protein_substitution(annotation[str], score[str])
```

In [22]:
df.select("id", protein_substitution("annotation", "polyphen").alias("polyphen"), protein_substitution("annotation", "sift")\
    .alias("sift")).where("polyphen[0] >= 0").show()

+---------------+--------------+------------+
|             id|      polyphen|        sift|
+---------------+--------------+------------+
|22:25024098:G:A|[0.124, 0.581]| [0.0, 0.12]|
|22:28378810:T:G|[0.005, 0.005]|[0.37, 0.37]|
|22:29885644:C:A|[0.001, 0.001]|  [1.0, 1.0]|
|22:32831711:C:G|[0.027, 0.351]|[0.54, 0.78]|
+---------------+--------------+------------+

